In [1]:
from openai import OpenAI
import os
import pandas as pd

In [ ]:
""" Example item:
Item 3
Level c: 
Lisa erzählt, welche Geschichte über Hanna sie ärgerlich fand.

Q1: Kann man den Satz so verstehen, dass Lisa eine Geschichte ärgerlich fand?
Q2: Kann man den Satz so verstehen, dass Hanna eine Geschichte ärgerlich fand?

Answer Participant 1 Q1: Ja
Answer Participant 1 Q2: Nein
"""

df = pd.read_csv("data/exp8_materials_items.csv", sep=";", engine="python", names=["item", "condition", "phrase", "Q1", "Q2"])

condition_a = df[(df["condition"] == "a")]
print(condition_a)


In [3]:

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "YOUR_KEY"))

model_type = "gpt-4"

def get_completion(prompt):

    response = client.chat.completions.create(
        model=model_type,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0,
    )
    return response.choices[0].message.content

In [4]:
instruction_gen = """ 
Beispielsatz: "Maria hat Anna besucht, weil sie nett ist."

In diesem Beispielsatz kann man das Pronomen "sie" so interpretieren, dass es sich sowohl auf Maria als auch Anna bezieht. Es gibt andere Beispielsätze, da bezieht sich das Pronomen nur auf eine der genannten Personen. Für das folgende Experiment sollen alle möglichen Interpretationen vorsichtig erwogen werden.

"""

instruction_subj = """ 
Satz: "Maria hat Anna besucht, weil sie nett ist."
Kann man den Satz so verstehen, dass Maria nett ist? 
Die Antwort ist "Ja"
Es gibt Sätze, bei denen ist die Antwort "Nein"

"""

instruction_obj = """ 
Satz: "Maria hat Anna besucht, weil sie nett ist."
Kann man den Satz so verstehen, dass Anna nett ist? 
Die Antwort ist "Ja"
Es gibt Sätze, bei denen ist die Antwort "Nein"

"""

In [ ]:
results = []
conditions = ["a", "b", "c", "d", "e", "f", "g", "h"]
condition_levels = {
    "a": ["object", "argument", "in situ"],
    "b": ["object", "argument", "moved"],
    "c": ["object", "adjunct", "in situ"],
    "d": ["object", "adjunct", "moved"],
    "e": ["subject", "argument", "in situ"],
    "f": ["subject", "argument", "moved"],
    "g": ["subject", "adjunct", "in situ"],
    "h": ["subject", "adjunct", "moved"]
}
iterations = 0
participant = 1

while iterations < 2:
    for condition in conditions:

        df_con = df[(df["condition"] == condition)]

        for item, content, question_1, question_2 in zip(df_con["item"], df_con["phrase"], df_con["Q1"], df_con["Q2"]):

            prompt_1 = f"""
            Es ist folgender Satz gegeben.

            Satz: {content}

            {question_1}
            
            Antworte nur mit "Ja" oder "Nein"
            """

            response_1 = get_completion(prompt_1)

            if response_1 == "Ja":
                value = 1
            else:
                value = 0

            results.append([int(item),
                            participant,
                            condition,
                            condition_levels[condition][0],
                            condition_levels[condition][1],
                            condition_levels[condition][2], 
                            content, 
                            "Q1", 
                            question_1, 
                            response_1,
                            value])

            prompt_2 = f"""
            Es ist folgender Satz gegeben.

            Satz: {content}

            {question_2}
            
            Antworte nur mit "Ja" oder "Nein"
            """

            response_2 = get_completion(prompt_2)

            if response_2 == "Ja":
                value = 1
            else:
                value = 0

            results.append([int(item),
                            participant, 
                            condition, 
                            condition_levels[condition][0],
                            condition_levels[condition][1],
                            condition_levels[condition][2],
                            content, 
                            "Q2", 
                            question_2, 
                            response_2,
                            value])
    
    iterations += 1

df_results = pd.DataFrame(results, columns=["item",
                                            "participant", 
                                            "condition",
                                            "phrase", 
                                            "argadj",
                                            "mov",
                                            "content",
                                            "question_no", 
                                            "question", 
                                            model_type,
                                            "value"])
print(df_results.head)
df_results.to_csv('results/gpt4_results.csv', index=False)